In [ ]:
library(GEOquery)

In [3]:
my_id <- "GSE178913"
gse_la <- getGEO(my_id)

Found 1 file(s)

GSE178913_series_matrix.txt.gz



In [10]:
gse <- gse_la[[1]]

In [7]:
gse <- read.table(
    'data/GSE178913_series_matrix.txt.gz',
    sep = '\t', header = T,
    comment.char = '!'
)

In [11]:
gse

ExpressionSet (storageMode: lockedEnvironment)
assayData: 0 features, 10 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM5400796 GSM5400797 ... GSM5400805 (10 total)
  varLabels: title geo_accession ... tissue:ch1 (39 total)
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation: GPL11154 